In [ ]:
import sys
sys.path.append('C:\\python_modules')
import monitor_tools_wb, bf_creds
from monitor_tools_resources import brand_variations_dict
from monitor_tools_resources import name_similarity_both_true, name_similarity_both_true_several_words, name_similarity_both_true_exact_words

from random import randint
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
import pickle
import re
import numpy as np
from time import sleep
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, Text, DateTime, Float, Date

chrome_driver_path = 'C:\chromedriver_win32\chromedriver.exe'

pd.set_option('display.max_columns', None)

## Получение информации по нашим товарам в наличии (airflow)

In [ ]:
%%time
# получение нашего стока из базы
sql_query = '''
                SELECT
                    "nmId" AS nm_id,
                    article AS article
                FROM wb_products_info
                WHERE stock > 0
            '''

engine = create_engine(bf_creds.postgre_user_engine, echo=True)
try:
    our_stock = pd.read_sql(sql_query, con=engine)
finally:
    engine.dispose()

our_stock['url'] = 'https://www.wildberries.ru/catalog/' + our_stock.nm_id + '/detail.aspx'

our_stock_with_price = monitor_tools_wb.parse_product_cards_data(our_stock, chrome_driver_path, param_json_src=True)

our_stock_with_price_and_json_data = monitor_tools_wb.parse_product_cards_data_json(our_stock_with_price)

with open('wb_files/pkl/wb_our_stock_data.pkl', 'wb') as file:
    pickle.dump(our_stock_with_price_and_json_data, file)

## Получение списка товаров по списку урлов категорий (airflow)

In [ ]:
wb_category_url_list = [
            'https://www.wildberries.ru/catalog/sport/sportivnoe-pitanie?sort=popular&page=1',
            'https://www.wildberries.ru/catalog/pitanie/napitki?sort=popular&page=1&xsubject=3382%3B3441',
            'https://www.wildberries.ru/catalog/pitanie/sneki?sort=popular&page=1&xsubject=3014',
            'https://www.wildberries.ru/catalog/pitanie/sladosti/shokolad-i-shokoladnye-batonchiki?sort=popular&page=1&xsubject=2878',
            'https://www.wildberries.ru/catalog/pitanie/sladosti/shokolad-i-shokoladnye-batonchiki?sort=popular&page=1&xsubject=3568',
            'https://www.wildberries.ru/catalog/produkty/dobavki-pishchevye?sort=popular&page=1',
            'https://www.wildberries.ru/catalog/produkty/sladosti/orehi?sort=popular&page=1&xsubject=3407',
            'https://www.wildberries.ru/catalog/produkty/sladosti/pasty?sort=popular&page=1',
            'https://www.wildberries.ru/catalog/dom-i-dacha/zdorove/vitaminy-i-bady?sort=popular&page=1',
            'https://www.wildberries.ru/catalog/zdorove/lechebnoe-pitanie-?sort=popular&page=1'
]

prod_list = monitor_tools_wb.get_product_list_from_cat_pages(wb_category_url_list, chrome_driver_path, 14000)

with open('wb_files/pkl/wb_prod_list.pkl', 'wb') as file:
    pickle.dump(prod_list, file)

## Получение информации из джейсонов карточек товара (airflow)

In [ ]:
%%time

with open('wb_files/pkl/wb_prod_list.pkl', 'rb') as file: 
    prod_list = pickle.load(file)
    
prod_list_with_json_data = monitor_tools_wb.parse_product_cards_data_json(prod_list)

with open('wb_files/pkl/wb_prod_list_data.pkl', 'wb') as file:
    pickle.dump(prod_list_with_json_data, file)

# Сопоставление

#### Функции для задачи

In [ ]:
# функция для перевода строковых значений разного формати в int
def to_int(x):
    formatted_value = re.sub("\..*", "", x)
    formatted_value = re.sub("\/.*", "", formatted_value)
    formatted_value = re.sub("[^0-9]", "", formatted_value)
    
    if formatted_value == '':
        return 0

    return int(formatted_value)

#### Загрузка данных

In [ ]:
with open('wb_files/pkl/wb_prod_list_data.pkl', 'rb') as file: 
    data_wb = pickle.load(file)
    
with open('wb_files/pkl/wb_our_stock_data.pkl', 'rb') as file: 
    data_our = pickle.load(file)
    
data_our = data_our[data_our['price_with_disc'] > 0]
data_our.reset_index(inplace=True)
data_our.drop(columns=['index'], inplace=True)
    
print(data_wb.shape, data_our.shape)

#### Предобработка

In [ ]:
%%time

engine = create_engine(bf_creds.postgre_user_engine, echo=True)

# удаление из data_wb товаров, которые уже на мониторинге
query = '''
            SELECT id
            FROM monitor_mp_prices
        '''
ids_in_monitoring = pd.read_sql(query, con=engine)
data_wb.drop(data_wb[data_wb.nm_id.isin(ids_in_monitoring.id)].index, inplace=True)

# удаление наших товаров из списка товаров конкурентов
data_wb.drop(data_wb[data_wb.nm_id.isin(data_our.nm_id)].index, inplace=True)

# замена nan пустую строку
columns_with_str_nan = ['selling.brand_name', 'Страна производства', 'Вкус']
for col_name in columns_with_str_nan:
    data_our[col_name].fillna('', inplace=True)
    data_wb[col_name].fillna('', inplace=True)

# дроп строк, где не хватает важных данных
data_our.dropna(subset=['imt_name'], inplace=True)
data_wb.dropna(subset=['imt_name'], inplace=True)

# замена ; на , (это для нормального отображения данных в csv)
for column in ['Вкус', 'imt_name']:
    data_our[column] = data_our[column].apply(lambda x: x.replace(';', ','))
    data_wb[column] = data_wb[column].apply(lambda x: x.replace(';', ','))

# приведение названий брендов к единому формату (используется словарь определенный ранее)
for key in brand_variations_dict:
    for name in brand_variations_dict[key]:
        data_our['selling.brand_name'] = data_our['selling.brand_name'].apply(lambda x: x.replace(name, key) if x == name else x)
        data_wb['selling.brand_name'] = data_wb['selling.brand_name'].apply(lambda x: x.replace(name, key) if x == name else x)

data_wb['selling.brand_name'] = data_wb['selling.brand_name'].apply(lambda x: 'California Gold Nutrition' if 'California Gold' in x else x)
        
# приведение данных в колонках важных для сопоставления к единому числовому формату с помощью ранее написанной функции to_int (оставляет в колонках только цифры, если цифр нет, то 0)
search_columns = ['Количество предметов в упаковке', 
                  'Вес товара с упаковкой (г)',
                  'Вес товара без упаковки (г)',
                  'Ширина упаковки',
                  'Высота упаковки',
                  'Глубина упаковки',
                  'Пищевая ценность белки',
                  'Пищевая ценность жиры',
                  'Пищевая ценность углеводы',
                  'Энергетическая ценность калории (на 100 гр.)']

for col_name in search_columns:
    if col_name in data_our.columns:
        data_our[col_name].fillna('0', inplace=True)
        data_our[col_name] = data_our[col_name].apply(to_int)
        
    if col_name in data_wb.columns:
        data_wb[col_name].fillna('0', inplace=True)
        data_wb[col_name] = data_wb[col_name].apply(to_int)
        
engine.dispose()

#### Сопоставление

In [ ]:
%%time

# удаление из wb_data колонок, которые не будут участвовать в сопоставлении
search_lake = data_wb[['nm_id', 
                       'imt_name',
                       'selling.brand_name',
                       'price_with_disc',
                       'Количество предметов в упаковке',
                       'Вкус',
                       'url']]

# первый этап - сопоставление товаров по параметрам
results_1 = pd.DataFrame()

for i in range(data_our.shape[0]):
    
    # фильтр по бренду
    search_pool = search_lake[search_lake['selling.brand_name'] == data_our.iloc[i]['selling.brand_name']]
    
    # фильтр по количеству в упаковке
    if data_our.iloc[i]['Количество предметов в упаковке'] != 0:
        search_pool = search_pool[(search_pool['Количество предметов в упаковке'] == data_our.iloc[i]['Количество предметов в упаковке']) 
                                  | (search_pool['Количество предметов в упаковке'] == 0)]
    
    # фильтр по цене (+-30% и еще сверху +-50 рублей, чтобы нормально учитывать разницу у товаров с низкими ценами)
    if data_our.iloc[i]['price_with_disc'] != 0:
        search_pool = search_pool[(search_pool['price_with_disc'] <= (int(data_our.iloc[i]['price_with_disc'] * 1.3) + 50)) & 
                                  (search_pool['price_with_disc'] >= (int(data_our.iloc[i]['price_with_disc'] * 0.7) - 50))]

    # добавление к каждой строке нашего соответствующего товара и присоединение к результирующему датасету
    search_pool['our_article'] = data_our.iloc[i]['article']
    search_pool['our_nm_id'] = data_our.iloc[i]['nm_id']
    search_pool['our_url'] = data_our.iloc[i]['url']
    search_pool['our_name'] = data_our.iloc[i]['imt_name']
    search_pool['our_price'] = data_our.iloc[i]['price_with_disc']

    results_1 = results_1.append(search_pool)
    results_1.reset_index(inplace=True)
    results_1.drop(columns='index', inplace=True)
    
print(results_1.shape)

In [ ]:
results_1

In [ ]:
%%time
# удаление тех пар, которые уже сопоставлялись до этого

engine = create_engine(bf_creds.postgre_user_engine)
query = '''
            SELECT DISTINCT(CONCAT(our_nm_id,nm_id)) as concat_ids
            FROM monitor_wb_already_compared_pairs
        '''
already_compared = pd.read_sql(query, con=engine)
engine.dispose()

concat_ids_lst = list(already_compared.concat_ids)

results_1['already_compared'] = results_1.apply(lambda x: True if (x['our_nm_id'] + x['nm_id']) in concat_ids_lst else False, axis=1)
results_1 = results_1[results_1['already_compared']==False]

results_1.reset_index(inplace=True)
results_1.drop(columns='index', inplace=True)

In [ ]:
%%time

# второй этап фильтрации - текстовые фильтры
results_1['name_similarity_index'] = 0
results_1['searchable_name'] = False

for i in range(results_1.shape[0]):
    searchable_name = False
    similarity_index = 0
    
    # по точному вхождению слов в оба названия
    for sublist in name_similarity_both_true_exact_words:
        our_name_contains = any((word in re.sub("[^А-Яа-яA-za-z0-9 ]", " ", results_1['our_name'][i]).lower().split() for word in sublist))
        comp_name_contains = any((word in re.sub("[^А-Яа-яA-za-z0-9 ]", " ", results_1['imt_name'][i]).lower().split() for word in sublist))

        if our_name_contains:
            searchable_name = True
        if our_name_contains and comp_name_contains:
            similarity_index += 1
            break
                
    # по вхождениям подстрок в оба названия
    if similarity_index == 0:
        for sublist in name_similarity_both_true:
            our_name_contains = any((word in results_1['our_name'][i].lower() for word in sublist))
            comp_name_contains = any((word in results_1['imt_name'][i].lower() for word in sublist))

            if our_name_contains:
                searchable_name = True
            if our_name_contains and comp_name_contains:
                similarity_index += 1
                break
            
    # по вхождениям комбинации подстрок в оба названия (для названий, где не нашлось)
    if similarity_index == 0:  
        for sublist in name_similarity_both_true_several_words:
            if similarity_index:
                break

            our_name_contains = False
            comp_name_contains = False

            for words_list in sublist:
                if not our_name_contains:
                    if all([word in results_1['our_name'][i].lower() for word in words_list]):
                        our_name_contains = True
                        searchable_name = True
                if not comp_name_contains:
                    if all([word in results_1['imt_name'][i].lower() for word in words_list]):
                        comp_name_contains = True  
                if our_name_contains and comp_name_contains:
                    similarity_index += 1
                    break
                    
    # присвоение результирующего индекса похожести и того есть ли в нашем названии вообще вхождения поисковых слов из списков                  
    results_1['name_similarity_index'][i] = similarity_index
    results_1['searchable_name'][i] = searchable_name
    
results_2 = results_1[(results_1['searchable_name'] == False) | 
                      ((results_1['searchable_name'] == True) & (results_1['name_similarity_index'] > 0))]

In [ ]:
# датафрейм с товарами, которые отсеял фильтр (для проверки корректности его работы)
results_2_false = results_1[((results_1['searchable_name'] == True) & (results_1['name_similarity_index'] == 0))]

In [ ]:
results_2_false

In [ ]:
# сохранение в csv (для финальной ручной сверки)
results_2[['our_article',
            'our_nm_id',
            'our_name',
            'our_price',
            'our_url',
            'nm_id',
            'imt_name',
            'Количество предметов в упаковке',
            'Вкус',
            'price_with_disc',
            'url',
            'selling.brand_name',
            'name_similarity_index',
            'searchable_name']].to_csv('wb_files/csv/results_2.csv', encoding='utf-8-sig', sep='|', index=False)

In [ ]:
results_2_false[['our_article',
                'our_nm_id',
                'our_name',
                'our_price',
                'our_url',
                'nm_id',
                'imt_name',
                'Количество предметов в упаковке',
                'Вкус',
                'price_with_disc',
                'url',
                'selling.brand_name',
                'name_similarity_index',
                'searchable_name']].to_csv('wb_files/csv/results_2_false.csv', encoding='utf-8-sig', sep='|', index=False)

## Загрузка результатов ручной сверки в бд с предварительным обновлением цены и продавца

In [ ]:
# загрузка эксель файла с результатами сверки
search_results_final = pd.read_excel('wb_files/wb_search_results_final.xlsx', dtype={'our_article': 'str',
                                                                            'our_nm_id': 'str',
                                                                            'nm_id': 'str'})

search_results_final = search_results_final[['our_article', 'our_nm_id', 'our_name', 'our_price', 'our_url', 'nm_id', 'imt_name', 'price_with_disc', 'url']]

search_results_final['mp'] = 'WB'

search_results_final.rename(columns={'nm_id': 'id', 
                                     'our_nm_id': 'our_id',
                                     'imt_name': 'name'}, inplace=True)

# актуализация цены товаров
search_results_final_with_seller = monitor_tools_wb.parse_product_cards_data(search_results_final, 
                                                                             chrome_driver_path, 
                                                                             param_seller_name=True)

# загрузка в таблицу в бд, которая используется для bi
engine = create_engine(bf_creds.postgre_user_engine)

search_results_final_with_seller.to_sql(
         'monitor_mp_prices',
         engine,
         if_exists='append',
         index=False)

engine.dispose()

## Загрузка в бд пар id товаров, которые уже были сопоставлены

In [ ]:
# в отдельную эксельку из файла, в котором проводилась сверка, выносятся два столбца: наш id товара и id товара конкурента, который скрипт сопоставления ему сопоставил
# эти пары загружаются в базу данных, чтобы при следующей сверке их не учитывать

#### тут надо переписать, чтобы дата добавлялась в колонку "added_at"

already_compared_pairs = pd.read_excel('wb_files/wb_already_compared_pairs.xlsx', dtype={'our_nm_id': 'str', 'nm_id': 'str'})
already_compared_pairs['added_at'] = date.today()

engine = create_engine(bf_creds.postgre_user_engine, echo=True)

already_compared_pairs.to_sql(
         'monitor_wb_already_compared_pairs',
         engine,
         if_exists='append',
         index=False)

engine.dispose()